## **SETD2 with SAM, complexed pose**

## **Import modules**

In [1]:
import os
import sys

data_path=os.path.abspath('/media/dataHog/hca/SETD2') #Where your group data is
base_path=os.path.abspath('/media/dataHog/hca/SimFound_v2/source/') #Where your source code is (SFv2)

sys.path.append(base_path)
sys.path.append(data_path)

import importlib

import warnings
warnings.filterwarnings('ignore')
import Protocols as P

import main
import tools
import Featurize as F


from simtk.unit import *

some modules not loaded


## **Define Project**

In [3]:
importlib.reload(main)

workdir=data_path
results=workdir+'/results'
inputs=workdir+'inputs/structures/'

workdir=tools.Functions.fileHandler([workdir], _new=False)
tools.Functions.fileHandler([results, inputs])


#protein=['WT', 'R167Q', 'I602G']
protein=['setd2_complexed']

ligand=['SAM']
parameters=['310K']
timestep=20*picoseconds

project=main.Project(title='SETD2-SAM', 
                     hierarchy=('protein', 'ligand', 'parameter'), 
                     workdir=workdir,
                     parameter=parameters, 
                     replicas=4, 
                     protein=protein, 
                     ligand=ligand, 
                     results=results,
                     topology='wt_SETD2_complexed.pdb',
                     timestep=5*picoseconds,
                    initial_replica=3)


project_systems=project.setSystems()

Converted parameter "temperature" (in K) into scalar: 310.0 K
System defined: ('setd2_complexed', 'SAM', '310K', '3'), ID: 310.0 K
System defined: ('setd2_complexed', 'SAM', '310K', '4'), ID: 310.0 K


## **Simulation Protocol**

## Set defaults

In [4]:
std_report=timestep
print(std_report)

SETD2_variants={('A',1499): 'CYX',
                ('A',1501): 'CYX',
                ('A',1516): 'CYX',
                ('A',1520): 'CYX',
                ('A',1529): 'CYX',
                ('A',1533):'CYX',
                ('A',1539):'CYX',
                ('A',1631):'CYX',
                ('A',1678):'CYX',
                ('A',1680):'CYX',
                ('A',1685):'CYX'}

NPT_equilibration={'ensemble': 'NPT',
                'step': 5*nanoseconds, 
                'report': std_report, 
                'restrained_sets': {'selections': ['protein and backbone', 'resname ZNB', 'resname SAM'],
                                    'forces': [100*kilojoules_per_mole/angstroms, 150*kilojoules_per_mole/angstroms, 150*kilojoules_per_mole/angstroms]}}

NPT_production={'ensemble': 'NPT',
                'step': 500*nanoseconds, 
                'report': std_report}

20 ps


## Setup simulation protocol

In [5]:
importlib.reload(P)
importlib.reload(tools)
extra_pdb = ['ZNB_open.pdb', 'SAM.pdb']
#smiles = 'C[S+](CC[C@@H](C(=O)[O-])N)C[C@@H]1[C@H]([C@H]([C@@H](O1)N2C=NC3=C(N=CN=C32)N)O)O'
xml_files = ['protein.ff14SB.xml', 'tip4pew.xml', 'ZNB.xml', 'SAM.xml', 'bonded.xml', 'gaff.xml']

gpu_index='1'


simulated_systems={}
for name, system in project_systems.items():

    if system.ligand == 'SAM':
    
        sim_tools=P.Protocols(workdir=system.path, 
                              project_dir=project.workdir,
                              def_input_struct=project.def_input_struct,
                              def_input_ff=project.def_input_ff)
    
        system_omm=sim_tools.pdb2omm(input_pdb=system.input_topology,
                                     extra_input_pdb=extra_pdb,
                                     ff_files=xml_files,
                                     protonate=True,
                                     solvate=True,
                                     pH_protein=7.0,
                                     name=name,
                                     residue_variants=SETD2_variants)
                                    #insert_molecules=True)
                                    #insert_smiles=smiles)
            
        system_omm.setSimulations(dt=0.002*picoseconds, 
                                  temperature = system.scalar,
                                  pH=7.0,
                                  friction = 1/picosecond,
                                  equilibrations=[NPT_equilibration],
                                  productions=[NPT_production],
                                  pressure=1*atmospheres,
                                  gpu_index=gpu_index)
                                      
        simulated_systems[name]=system_omm


Setting openMM simulation protocols in /media/dataHog/hca/SETD2/setd2_complexed-SAM-310K/replicate3.
	Adding extra PDB file 1 to pre-system: /media/dataHog/hca/SETD2/inputs/structures/ZNB_open.pdb
	<Topology; 1 chains, 3 residues, 15 atoms, 12 bonds>
	Adding extra PDB file 2 to pre-system: /media/dataHog/hca/SETD2/inputs/structures/SAM.pdb
	<Topology; 1 chains, 1 residues, 49 atoms, 51 bonds>
	Added 1: /media/dataHog/hca/SETD2/inputs/structures/ZNB_open.pdb
	Added 2: /media/dataHog/hca/SETD2/inputs/structures/SAM.pdb
	Adding extra FF file 1: /media/dataHog/hca/SETD2/inputs/forcefields/protein.ff14SB.xml
	Adding extra FF file 2: /media/dataHog/hca/SETD2/inputs/forcefields/tip4pew.xml
	Adding extra FF file 3: /media/dataHog/hca/SETD2/inputs/forcefields/ZNB.xml
	Adding extra FF file 4: /media/dataHog/hca/SETD2/inputs/forcefields/SAM.xml
	Adding extra FF file 5: /media/dataHog/hca/SETD2/inputs/forcefields/bonded.xml
	Adding extra FF file 6: /media/dataHog/hca/SETD2/inputs/forcefields/gaff.

## **WARNING!** Run simulation(s)

In [ ]:
simulations={}
broken=[]
for name, simulation in simulated_systems.items():
        
    try:
        print(f'Simulating: {name}')
        simulation.runSimulations(run_productions=True)
    
        simulations[name]=simulation
    except ValueError:
        print('Simulation failed. Error probably related with energy values')
        broken.append(name)
        pass
    except:
        print('This is bat country.')

Simulating: setd2_complexed-SAM-310K-3
Spwaning new simulation:
	Energy minimization:
	System potential energy: -219799.9789289683 kJ/mol
	PDB file generated: /media/dataHog/hca/SETD2/setd2_complexed-SAM-310K/replicate3/minimization.pdb
	System potential energy: -1279417.6911244383 kJ/mol
EQ run 1: NPT
Spwaning new simulation:
	equilibration_NPT-1: populating file /media/dataHog/hca/SETD2/setd2_complexed-SAM-310K/replicate3/equilibration_NPT-1.
	Applying custom external force:  	(k/2)*periodicdistance(x, y, z, x0, y0, z0)^2
	Restraining set (100 kJ**2/(A**3 mol**2)): 1080
	Restraining set (150 kJ**2/(A**3 mol**2)): 15
	Restraining set (150 kJ**2/(A**3 mol**2)): 49
	Adding MC barostat for equilibration_NPT-1 ensemble
	First run equilibration_NPT-1. Assigning velocities to 310.0 K
	PDB file generated: /media/dataHog/hca/SETD2/setd2_complexed-SAM-310K/replicate3/equilibration_NPT-1_0.pdb
	equilibration in NPT ensemble (5000.0 ps)...
	No checkpoint files found. Starting new.


In [ ]:
simulations